In [1]:
!pip install azure-cognitiveservices-vision-customvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 13.1 MB/s eta 0:00:00


In [2]:
ENDPOINT = "https://westeurope.api.cognitive.microsoft.com/"
training_key = "cf5b66245476417180cfe7052794c0c4"

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid
from math import ceil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os


In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [5]:
path_dict = dict()
SKIP_TAGS = ["New Plant Diseases Dataset(Augmented)", "valid", "train", "Blueberry___healthy", "Orange___Haunglongbing_(Citrus_greening)", "Raspberry___healthy", "Soybean___healthy"]
for dirname, _, filenames in os.walk('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'):
    tag_name = dirname.split("/")[-1]
    if tag_name in SKIP_TAGS:
        continue
    if tag_name not in path_dict:
        path_dict[tag_name] = []
    for file_name in filenames:
        with open(os.path.join(dirname, file_name), "rb") as image_contents:
            path_dict[tag_name].append(image_contents.read())

In [6]:
for tag in path_dict.keys():
    tag_size = len(path_dict[tag]) // 33
    project = trainer.create_project(tag)
    image_list = []
    tag_obj = trainer.create_tag(project.id, tag)
    for img in path_dict[tag]:
        image_list.append(ImageFileCreateEntry(name=uuid.uuid4(), contents=img, tag_ids=[tag_obj.id]))
    tag_list = [tg for tg in path_dict.keys() if tg != tag]
    other_tag_obj = trainer.create_tag(project.id, "other")
    for other_tag in tag_list:
        randomly_selected = random.sample(path_dict[other_tag], tag_size)
        image_list.extend([ImageFileCreateEntry(name=uuid.uuid4(), contents=img, tag_ids=[other_tag_obj.id]) for img in randomly_selected])
    MAX_BATCH_SIZE = 64
    num_batches = ceil(len(image_list) / MAX_BATCH_SIZE)
    for i in range(num_batches):
        start_index = i * MAX_BATCH_SIZE
        end_index = min((i + 1) * MAX_BATCH_SIZE, len(image_list))
        batch_images = image_list[start_index:end_index]
        trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=batch_images))

In [7]:
path_dict = dict()
SKIP_TAGS = ["New Plant Diseases Dataset(Augmented)", "valid", "train", "Blueberry___healthy", "Orange___Haunglongbing_(Citrus_greening)", "Raspberry___healthy", "Soybean___healthy"]
for dirname, _, filenames in os.walk('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'):
    tag_name = dirname.split("/")[-1]
    if tag_name in SKIP_TAGS:
        continue
    if tag_name not in path_dict:
        path_dict[tag_name] = []
    for file_name in filenames:
        with open(os.path.join(dirname, file_name), "rb") as image_contents:
            path_dict[tag_name].append(image_contents.read())

In [8]:
for project in trainer.get_projects():
    tags = trainer.get_tags(project.id)
    tag_obj = tags[0] if tags[0].name != "other" else tags[1]
    tag_size = len(path_dict[tag_obj.name]) // 33
    image_list = []
    for img in path_dict[tag_obj.name]:
        image_list.append(ImageFileCreateEntry(name=uuid.uuid4(), contents=img, tag_ids=[tag_obj.id]))
    tag_list = [tg for tg in path_dict.keys() if tg != tag]
    other_tag_obj = tags[0] if tags[0].name == "other" else tags[1]
    for other_tag in tag_list:
        randomly_selected = random.sample(path_dict[other_tag], tag_size)
        image_list.extend([ImageFileCreateEntry(name=uuid.uuid4(), contents=img, tag_ids=[other_tag_obj.id]) for img in randomly_selected])
    MAX_BATCH_SIZE = 64
    num_batches = ceil(len(image_list) / MAX_BATCH_SIZE)
    for i in range(num_batches):
        start_index = i * MAX_BATCH_SIZE
        end_index = min((i + 1) * MAX_BATCH_SIZE, len(image_list))
        batch_images = image_list[start_index:end_index]
        trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=batch_images))